In [9]:
from jp_doodle import dual_canvas
from IPython.display import display

color_chooser = dual_canvas.SnapshotCanvas("color_chooser.png", width=600, height=600)
color_chooser.display_all()

color_chooser.js_init("""

var color_chooser = function(config) {
    var settings = $.extend({
        side: 200,
        x: 0,
        y: 0,
        font: "normal 10px Arial",
        callback: null,
    }, config);
    const color_max = 255;
    // intensity is total of all colors
    const max_intensity = color_max * 3;
    var side = settings.side;
    var current_intensity = max_intensity;
    var current_rgb = [color_max, color_max, color_max];

    var right = side * 0.25;
    var left = side - right;
    var top = left;
    var bottom = right;
    
    // intensity frame shows intensities
    slider_height = 20;
    column_width = 0.25;
    
    var intensity_frame = element.frame_region(
        left, 0, side, side, 0, 0, 1.0, max_intensity+slider_height);
    // scaling factors for width and height
    var i_wscale = right;
    var i_hscale = side / (max_intensity+slider_height)
    var gray_scale = function(intensity) {
        var third = intensity * 0.333;
        return element.array_to_color([third, third, third]);
    };
    for (var intensity=0; intensity<=max_intensity; intensity++) {
        intensity_frame.rect({
            x: column_width,
            y: intensity,
            w: i_wscale * column_width,
            h: i_hscale * 1,
            color: gray_scale(intensity)
        });
    }
    intensity_frame.rect({
        name: "islider",
        x: 0,
        y: current_intensity,
        w: i_wscale * column_width * 3,
        h: i_hscale * slider_height,
        color: gray_scale(current_intensity),
    });
    intensity_frame.rect({
        name: "ibox",
        x: 0,
        y: current_intensity,
        w: i_wscale * column_width * 3,
        h: i_hscale * slider_height,
        fill: false,
        color: "black",
    });
    // invisible column for events
    intensity_frame.rect({
        name: "icolumn",
        color: "rgba(0,0,0,0)",
        //color: "rgba(255,0,0,1.0)", fill: false, // for debug.
        x: 0,
        y: 0,
        w: i_wscale * column_width * 3,
        h: i_hscale * max_intensity,
    });
    var intensity_mouse_over = function(event) {
        var frame_location = intensity_frame.event_model_location(event);
        var intensity = Math.round(frame_location.y);
        intensity = Math.min(Math.max(0, intensity), max_intensity);
        element.change("ibox", {y:intensity});
        element.change("islider", {y:intensity, color:gray_scale(intensity)});
        current_intensity = intensity;
        draw_triangle_frame();
    }
    element.on_canvas_event("mousemove", intensity_mouse_over, "icolumn");
    
    // color at current intensity level
    var color_array_at = function(r, g) {
        if ((r < 0) || (g < 0) || (r + g > 1.0)) {
            return null;
        }
        var b = 1 - r - g;
        var sr = Math.round(Math.min(255, current_intensity * r));
        var sg = Math.round(Math.min(255, current_intensity * g));
        var sb = Math.round(Math.min(255, current_intensity * b));
        //return element.array_to_color([sr, sg, sb]);
        return [sr, sg, sb];
    };
    
    // triangle frame shows color choices at the current intensity
    var triangle_frame = element.vector_frame(
        {x:left, y:0},
        {x:left*0.5, y:top},
        {x:0, y:bottom}
    );
    var draw_triangle_frame = function() {
        triangle_frame.reset_frame();
        // draw colors as circles
        var delta = 0.02
        var radius = top * delta;
        for (var r=0; r<=1; r+=delta) {
            for (var g=0; g<=1 - r; g+=delta) {
                var color_array = color_array_at(r, g);
                if (color_array) {
                    var color = element.array_to_color(color_array);
                    triangle_frame.circle({x:r, y:g, color:color, r:radius});
                }
            }
        }
        // mouse tracker circle (initially hidden)
        triangle_frame.circle({
            name:"color_track", fill: false, r:radius*3,
            x:0, y:0, color:"black", hide:true});
        // outline
        //triangle_frame.polygon({
        //    points: [[0,0], [0,1], [1,0]],
        //    fill: false
        //});
        var color_mouse = function(event) {
            var frame_location = triangle_frame.event_model_location(event);
            var r = frame_location.x;
            var g = frame_location.y;
            var color_array = color_array_at(r, g);
            if (color_array) {
                var color = element.array_to_color(color_array);
                element.change("color_track", {x:r, y:g, hide:false});
                element.change("preview", {color: color})
                element.change("rgb", {text: color})
                if (event.type == "click") {
                    // Color selected!
                    element.change("color_choice", {hide: false, text: color});
                    element.change("final", {hide: false, color: color});
                    if (settings.callback) {
                        settings.callback(color_array, color);
                    }
                }
            } else {
                element.change("color_track", {hide:true});
            }
        }
        // invisible triangle to receive events
        triangle_frame.polygon({
            points: [[0,0], [0,1], [1,0]],
            name: "color_choices",
            color: "rgba(0,0,0,0)",
        });
        element.on_canvas_event("mousemove", color_mouse, "color_choices");
        element.on_canvas_event("click", color_mouse, "color_choices");
    };
    draw_triangle_frame();
    // preview swatch
    var swatch_offset = bottom * 0.5;
    element.circle({name: "preview", x:swatch_offset, y:side-swatch_offset,
        r: swatch_offset, color: "white"})
    // outline
    element.circle({fill:false, x:swatch_offset, y:side-swatch_offset,
        r: swatch_offset, color: "black"})
    element.text({name: "rgb", x:swatch_offset, y:side-swatch_offset,
        font:settings.font, text:" ", valign:"center", align:"center",
        background: "white"});
    element.rect({name: "final", hide:true,
        x:0, y:0, w:left, h: bottom*0.8
    });
    element.text({name: "color_choice", x:left*0.5, y:bottom*0.4, 
        text: "click to select color",
        align: "center", valign: "center", background: "white"})
};

color_chooser();
element.fit(null, 30);

""")

Tab(children=(VBox(children=(SnapshotCanvas(status=u'Not yet rendered'), JSProxyWidget(status=u'Not yet rendered'))), HTML(value=u'<img src="color_chooser.png" id="snapshot_id_20_1536459894674"/>\n <div id="snapshot_id_21_1536459894674">color_chooser.png</div>')), _titles={u'1': 'Snapshot', u'0': 'Canvas'})

In [ ]:
color_chooser.print_status()